In [1]:
import os, shutil
import numpy as np
from angstrom import Molecule, Trajectory
from angstrom.visualize import show
from copy import deepcopy
from surface_scan import read_datafile, get_lammps_data_lines, change_coordinates, write_job_file, write_spring_input
import matplotlib.pyplot as plt
%matplotlib inline

## Surface Scan
This notebook generates LAMMPS input files for Umbrella Sampling of large organic molecules on metal surfaces.

The molecule is translated on top of the surface in x and y dimensions with given step size and attached to given position using a spring.
For each configuration a LAMMPS simulation is initialized.

<font color="#c1202d">**NOTE: Make sure you check the reference input and job submission files are correct!**</font>

### Set directories and reference files

In [ ]:
molname = 'DC_Cu110'                                                      # Name of the molecule
lammpsdir = '/home/kutay/Documents/git/Nanocar/surface-scan/docs/lammps'  # Directory to create LAMMPS files
srcdir = '/home/kutay/Documents/git/Nanocar/surface-scan/docs/ref'        # Source directory for reference input files
datafile = os.path.join(srcdir, molname, 'data.%s' % molname)             # Reference LAMMPS data file
infile = os.path.join(srcdir, molname, 'in.%s' % molname)                 # Reference LAMMPS input file
jobfile = os.path.join(srcdir, molname, 'job.%s' % molname)               # Reference job submission file
scandir = os.path.join(lammpsdir, molname)
os.makedirs(scandir, exist_ok=True)

### Select scanning location and step size
The molecule starts scanning the surface at the `start` position and the travels in x for `xdist` with `dx` and y for `ydist`  with `dy` increments.
The `buffer` is used to add additional distances to x and y for more accurate WHAM calculations.

In [ ]:
start =  [23.3805, 24.1629, 10]          # Molecule starting position
xdist = 2                                # Distance to be traveled in y-axis (use 0 for 1D)
ydist = 1.5                              # Distance to be traveled in x-axis (use 0 for 1D)
dx, dy = 0.2, 0.1                        # Step size
buffer = 0.2                             # Buffer distance

xpos = np.arange(start[0] - xdist - buffer, start[0] + xdist + buffer + dx, dx)
ypos = np.arange(start[1] - ydist - buffer, start[1] + ydist + buffer + dy, dy)
print('x: %i points | y: %i points | Total: %i points' % (len(xpos), len(ypos), len(xpos) * len(ypos)))

### Plot scan points
Here each blue point is a different initial configuration for a constrained MD simulation. The red points show the surface metal atoms.

In [ ]:
x_points, y_points = [], []
for yi, y in enumerate(ypos):
    for xi, x in enumerate(xpos):
        x_points.append(x)
        y_points.append(y)
surfatoms1 = [[21.582, 22.8912], [25.179, 22.8912], [21.582, 25.4346], [25.179, 25.4346]]
surfatoms2 = [23.3805, 24.1629]
plt.scatter(x_points, y_points, alpha=0.9)
plt.scatter([i[0] for i in surfatoms1], [i[1] for i in surfatoms1], c='r', alpha=0.9, s=200)
plt.scatter(surfatoms2[0], surfatoms2[1], c='r', alpha=0.5, s=200)

### Read molecule coordinates from LAMMPS data file
Read the data file and extract coordinates of the organic molecule and skip the surface atoms.
The coordinates of the molecule will be used to move it along the surface.

In [ ]:
N_SURF_ATOMS = 1400
atomids, atoms, coords = read_datafile(datafile, skip_atoms=N_SURF_ATOMS)
mol = Molecule(atoms=atoms, coordinates=coords)
print(mol)
# Center molecule to starting position to use as reference
mol.center(start)

nframes = int(len(xpos) * len(ypos))
natoms = len(coords)
# Write trajectory to make sure everything is right
traj = Trajectory()
traj.coordinates = np.empty((nframes, natoms, 3))
traj.atoms = np.empty((nframes, natoms), dtype='U2')
# Spring constants (x and y is changes for each simulation, z is selected from eq position)
SPRING = {'k': 200.0, 'xeq': 0.0, 'yeq': 0.0, 'zeq': 'NULL', 'r0': 0.0}

In [ ]:
# show(mol)

### Initialize LAMMPS files for Umbrella Sampling
For each starting x position, generate input files where the molecule moves in y direction.

In [ ]:
idx = 0
write_files = True
for xi, x in enumerate(xpos):
    for yi, y in enumerate(ypos):
        print('\r%i / %i | x: %.2f | y: %.2f' % (idx + 1, nframes, x, y), end='')
        # position the molecule
        newmol = Molecule(atoms=deepcopy(mol.atoms), coordinates=deepcopy(mol.coordinates))
        newmol.center([x, y, start[2]])
        if write_files:
            # initialize new directory
            simdir = os.path.join(scandir, '%i-%i' % (xi, yi))
            # simdir = os.path.join(scandir, '%i' % idx)
            os.makedirs(simdir, exist_ok=True)
            # create new data file
            newdatafile = os.path.join(simdir, 'data.%s' % molname)
            newlines = get_lammps_data_lines(newmol.coordinates, atomids, startid=N_SURF_ATOMS + 1)
            change_coordinates(datafile, newdatafile, newlines, skip_atoms=N_SURF_ATOMS)
            # add input file
            newinfile = os.path.join(simdir, 'in.%s' % molname)
            # shutil.copy(infile, newinfile)
            # Change spring coefficient (use only with spring)
            SPRING['xeq'], SPRING['yeq'] = x, y
            write_spring_input(infile, newinfile, SPRING, springindex=49)
            # add job file
            newjobfile = os.path.join(simdir, 'job.%s' % molname)
            write_job_file(jobfile, newjobfile, '%s-%i-%i' % (molname, xi, yi))
        # Save frame to trajectory
        traj.coordinates[idx] = newmol.coordinates
        traj.atoms[idx] = newmol.atoms
        idx += 1

In [ ]:
traj.write(os.path.join(lammpsdir, '%s-traj.xyz' % molname))